In [ ]:
import pandas as pd
import datetime
import time
from docx import Document
from docx.shared import Inches
from Funcs import letter_maker
import sqlite3

### Defintions (of the more confusing terms)
* Final_Val_Days - The number of days between the last day of investment data and the future deposit date for 
the ER. For example, if you obtained investment data until and including 05/31/2023 and the future deposit date for the ER is 07/12/23 – then the value should be: 30 + 12 = 42
* Most Recent Month/Year - This is the last full month we have data for. For example, if we have data until 05/31/2023, then the values are ‘May’ and ‘2023’ respectively 
* mostrecenteomthyr - This is a combination of monthrecentmth & mostrecentyr written as the last day of the month, for example - '05-31-2023'
* final_val_date - This is the future deposit date which is when the ER should deposit any corrective contribution investment earnings
* date_of_letter - Today's date
* Start/End Date - # Enter the beginning of quarter date for start date, and the beginning of quarter date for end date. After you enter these variables, it will automatically generate below
* single_part - This determines how many participants there are, valid entries are "Single" or "Multiple"
* single_part_first_name (and last) - If there is just a single participant, fill this out. Otherwise, leave blank (i.e. ' ')
* EE_ER - Enter "yes" if contains both EE and ER contributions, otherwise enter no
* neginvreturns - Enter "No" if there is no neginvreturns. Otherwise, enter anything else

In [ ]:
df_big = pd.read_csv("Plan.csv")
display(df_big)
Final_Val_Days = 53
Savings_Rate = '1.6%'
mostrecenteomthyr = '06-30-2023'
mostrecentmth = "June"
mostrecentyr = "2023"
plan_type = 'Thrift 401(k)' 
employer_name = "Plan, Inc."
old_ER = ''
final_val_date = '08-22-2023'
date_of_letter = '08142023' 
start_date = '10/01/2008'
end_date = '04/01/2023'
single_part = 'Single'
single_part_first_name = ''
single_part_last_name = ''
EE_ER = 'yes'
neginvreturns = "Yes"

### Pulling in Data from SQL

In [ ]:
starting_year = 10
ending_year = 23
er_number_with_type = ''

In [ ]:
from Funcs import retrieve_data, calculate_investment_earnings, format_percent, convert_to_dates, generate_dates_2

df = retrieve_data(starting_year, ending_year, er_number_with_type)

df_returns = df.copy()
df_returns = df_returns[["Name", "ER Number With Type", "FileName"]]
df_returns["Opening Balance"] = df["SumOfGA CLOS BAL EE"].shift() + df["SumOfGA CLOS BAL ER"].shift() + df["SumOfSA CLOS BAL EE"].shift() + df["SumOfSA CLOS BAL ER"].shift()
df_returns["Closing Balance"] = df["SumOfGA CLOS BAL EE"] + df["SumOfGA CLOS BAL ER"] + df["SumOfSA CLOS BAL EE"] + df["SumOfSA CLOS BAL ER"]
df_returns["Investment Earnings"] = df_returns.apply(lambda row: calculate_investment_earnings(row, df, df_returns), axis=1)
df_returns["Returns"] = ((df_returns["Investment Earnings"] * 2)/ (df_returns["Opening Balance"] + df_returns["Closing Balance"] - df_returns["Investment Earnings"]))
df_returns["Returns"] = df_returns["Returns"].apply(format_percent)

dates_df = df_returns["FileName"].apply(convert_to_dates)
df_returns[["Starting Dates", "Ending Dates"]] = dates_df
df_returns["Starting Dates"] = pd.to_datetime(df_returns["Starting Dates"])
df_returns["Ending Dates"] = pd.to_datetime(df_returns["Ending Dates"])
df_returns["Days"] = df_returns.apply(lambda row: generate_dates_2(row["Starting Dates"], row["Ending Dates"]), axis=1)

display(df_returns.head())
slicing_done = False

In [ ]:
df_returns = df_returns[["Starting Dates", "Ending Dates", "Days", "Opening Balance", "Returns"]]

if not slicing_done:
    df_returns = df_returns[1:]
    slicing_done = True
display(df_returns)
for i, value in enumerate(df_returns["Returns"]):
    if i < len(df_returns["Returns"]) - 1:
        print(f"'{value}',", end=" ")
    else:
        print(f"'{value}'", end="")

In [ ]:
df_big["VTD"] = pd.to_datetime(df_big["VTD"])  
df_big["Pay Date"] = pd.to_datetime(df_big["Pay Date"])
final_val_date = pd.to_datetime(final_val_date).strftime('%B %d, %Y')
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

from Funcs import generate_dates

output = generate_dates(start_date, end_date)
print("Start dates:", output[0])
print("End dates:", output[1])
print("Day differences:", output[2])

### Manual Rate Entry & Manual Adjustments 

In [ ]:
dfdata = pd.DataFrame({
    'Starting Dates': output[0],
    'Ending Dates': output[1],
    'Days': output[2],
    'Returns': ['2.22%', '3.87%', '-12.06%', '10.80%', '3.07%', '0.26%', '6.10%', '-18.08%', '16.75%', '5.77%', '12.28%', '3.90%', '4.95%', '-0.80%', '4.82%', '-6.30%', '-12.81%', '-3.50%', '5.71%', '5.08%', '4.21%']})
dfdata["Starting Dates"] = pd.to_datetime(dfdata["Starting Dates"])
dfdata["Ending Dates"] = pd.to_datetime(dfdata["Ending Dates"])
display(dfdata)

In [ ]:
# df = df_big[df_big["VTD"] == "12/20/2022"]
# df = df_big[(df_big["Pay Date"] == "11/10/2022") & (df_big["VTD"] == "12/06/2022")]
df = df_big
df = df.sort_values(by='Name', key=lambda x: x.str.split().str[0])
df.reset_index(drop=True, inplace=True)
display(df)

### Main Calculations

In [ ]:
start_time = time.time()
pd.options.mode.chained_assignment = None
 
from Funcs import EOQ, days_in_quarter, get_first_day_of_next_month
from Funcs import main_idea, main_idea_2

if EE_ER == "yes":
    calc_df = main_idea_2(df, dfdata, Savings_Rate, Final_Val_Days)
else:
    calc_df = main_idea(df, dfdata, Savings_Rate, Final_Val_Days)
    
df_for_later = df
sum_row = df.sum(numeric_only=True)
sum_df = pd.DataFrame([sum_row], columns=df.columns)
df = pd.concat([df, sum_df], ignore_index=True)
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")
display(df)
display(calc_df)

In [ ]:
if EE_ER == "yes":
    numeric_columns = ["EE Corrective Contribution", "EE Investment Earnings","ER Corrective Contribution",
    "ER Investment Earnings", "Total Corrective Contribution", "Total Investment Earnings"]
else:
    numeric_columns = ['Corrective Contribution', 'Investment Earnings']
df_determination = df.groupby(['Name'], as_index=False)[numeric_columns].sum(numeric_only=True)

if EE_ER == "yes":
    if len(df_determination[df_determination["Total Investment Earnings"] > 0]) >= 1:
        contains_positive = "yes"
    else: 
        contains_positive = "no"
    if len(df_determination[df_determination["Total Investment Earnings"] < 0]) >= 1:
        contains_negative = "yes"
    else: 
        contains_negative = "no"
else:
    if len(df_determination[df_determination["Investment Earnings"] > 0]) >= 1:
        contains_positive = "yes"
    else: 
        contains_positive = "no"
    if len(df_determination[df_determination["Investment Earnings"] < 0]) >= 1:
        contains_negative = "yes"
    else: 
        contains_negative = "no"

Single_VTD = df["VTD"][0].date()  

### Positive/Negative Procedure & Seperation 

In [ ]:
from Funcs import positive_procedure, negative_procedure, summary_procedure
from Funcs import positive_procedure_EE_ER, negative_procedure_EE_ER, summary_procedure_EE_ER

if EE_ER == "yes":
    if (contains_positive == "yes" and contains_negative == "yes"):
        df_positive = positive_procedure_EE_ER(df, Single_VTD)
        display(df_positive)
        df_negative = negative_procedure_EE_ER(df, Single_VTD)
        display(df_negative)
    else:
        df_summary = summary_procedure_EE_ER(df, Single_VTD, contains_negative)
        display(df_summary)
else:
    if (contains_positive == "yes" and contains_negative == "yes"):
        df_positive = positive_procedure(df, Single_VTD)
        display(df_positive)
        df_negative = negative_procedure(df, Single_VTD)
        display(df_negative)
    else:
        df_summary = summary_procedure(df, Single_VTD, contains_negative)
        display(df_summary)

### Letter Generation

In [ ]:
df_letter = df
df_letter.loc[len(df_letter)-1, "Name"] = "Total"
if contains_positive == "yes" and contains_negative == "yes":
    if EE_ER == "yes":
        df_positive_letter = df_letter[:-1]
        df_negative_letter = df_letter[:-1]
        df_positive_letter = df_letter[df_letter["Total Investment Earnings"] > 0]
        df_negative_letter = df_letter[df_letter["Total Investment Earnings"] < 0]
        df_positive_letter.loc[len(df_letter)-1, "Name"] = "Total"
        df_negative_letter.loc[len(df_letter)-1, "Name"] = "Total"

    else:
        df_positive_letter = df_letter[:-1]
        df_negative_letter = df_letter[:-1]
        df_positive_letter = df_letter[df_letter["Investment Earnings"] > 0]
        df_negative_letter = df_letter[df_letter["Investment Earnings"] < 0]
        df_positive_letter.loc[len(df_letter)-1, "Name"] = "Total"
        df_negative_letter.loc[len(df_letter)-1, "Name"] = "Total"
df_summary_letter = df_letter

In [ ]:
# (name1, name2, PlanType, er, ernum, fcm, fcy, corrcontr, invearn, mostrecent, vtd, accrual, garate, negreturns, SingleorMultiple):

SingleorMultiple = single_part
if single_part == 'Single':
    name1 = single_part_first_name
    name2 = single_part_last_name[:-5]
else:
    name1 = ''
    name2 = '  '
PlanType = plan_type
er = employer_name
ernum = old_ER
mostrecent = pd.Timestamp(mostrecenteomthyr)
vtd = df["VTD"][0].date()
accrual = pd.to_datetime(final_val_date)
garate = "3.35%"
negreturns = neginvreturns
# negreturns = "Yes"


# Case 1 (pos/neg)
if contains_positive == "yes" and contains_negative == "yes":
    fcm_pos = df_positive_letter["Pay Date"].min().strftime('%B')
    fcy_pos = int(df_positive_letter["Pay Date"].min().strftime('%Y'))
    if EE_ER == "yes": 
        corrcontr_pos = str(df_positive_letter[df_positive_letter["Name"] == "Total"]["Total Corrective Contribution"].item())
        invearn_pos = str(round(df_positive_letter[df_positive_letter["Name"] == "Total"]["Total Investment Earnings"].item(),2))
    else:
        corrcontr_pos = str(df_positive_letter[df_positive_letter["Name"] == "Total"]["Corrective Contribution"].item())
        invearn_pos = str(round(df_positive_letter[df_positive_letter["Name"] == "Total"]["Investment Earnings"].item(),2))
    fcm_neg = df_negative_letter["Pay Date"].min().strftime('%B')
    fcy_neg = int(df_negative_letter["Pay Date"].min().strftime('%Y'))
    if EE_ER == "yes": 
        corrcontr_neg = str(df_negative_letter[df_negative_letter["Name"] == "Total"]["Total Corrective Contribution"].item())
        invearn_neg = str(round(df_negative_letter[df_negative_letter["Name"] == "Total"]["Total Investment Earnings"].item(),2))
    else:
        corrcontr_neg = str(df_negative_letter[df_negative_letter["Name"] == "Total"]["Corrective Contribution"].item())
        invearn_neg = str(round(df_negative_letter[df_negative_letter["Name"] == "Total"]["Investment Earnings"].item(),2))
    letter_maker(name1, name2, PlanType, er, ernum, fcm_pos, fcy_pos, corrcontr_pos, invearn_pos, mostrecent, 
                 vtd, accrual, garate, negreturns, SingleorMultiple, 'Positive')
    letter_maker(name1, name2, PlanType, er, ernum, fcm_neg, fcy_neg, corrcontr_neg, invearn_neg, mostrecent, 
                 vtd, accrual, garate, negreturns, SingleorMultiple, 'Negative')
# Case 2 (summary)
else:
    fcm = df_summary_letter["Pay Date"].min().strftime('%B')
    fcy = int(df_summary_letter["Pay Date"].min().strftime('%Y'))
    if EE_ER == "yes": 
        corrcontr = str(df_summary_letter[df_summary_letter["Name"] == "Total"]["Total Corrective Contribution"].item())
        invearn = str(round(df_summary_letter[df_summary_letter["Name"] == "Total"]["Total Investment Earnings"].item(),2))
    else:
        corrcontr = str(df_summary_letter[df_summary_letter["Name"] == "Total"]["Corrective Contribution"].item())
        invearn = str(round(df_summary_letter[df_summary_letter["Name"] == "Total"]["Investment Earnings"].item(),2))
        
    letter_maker(name1, name2, PlanType, er, ernum, fcm, fcy, corrcontr, invearn, mostrecent, vtd, accrual, 
                 garate, negreturns, SingleorMultiple, 'Summary')
    
print(f"\033[1mname1\033[0m: {name1}, \033[1mname2\033[0m: {name2}")
print(f"\033[1mPlanType\033[0m: {PlanType}, \033[1mer\033[0m: {er}, \033[1mernum\033[0m: {ernum}")

if contains_positive == "yes" and contains_negative == "yes":
    print(f"\033[1mfcm_pos\033[0m: {fcm_pos}, \033[1mfcy_pos\033[0m: {fcy_pos}")
    print(f"\033[1mcorrcontr_pos\033[0m: {corrcontr_pos}, \033[1minvearn_pos\033[0m: {invearn_pos}")
    print(f"\033[1mfcm_neg\033[0m: {fcm_neg}, \033[1mfcy_neg\033[0m: {fcy_neg}")
    print(f"\033[1mcorrcontr_neg\033[0m: {corrcontr_neg}, \033[1minvearn_neg\033[0m: {invearn_neg}")
else:
    print(f"\033[1mfcm\033[0m: {fcm}, \033[1mfcy\033[0m: {fcy}")
    print(f"\033[1mcorrcontr\033[0m: {corrcontr}, \033[1minvearn\033[0m: {invearn}")

print(f"\033[1mmostrecent\033[0m: {mostrecent}, \033[1mvtd\033[0m: {vtd}, \033[1maccrual\033[0m: {accrual}")
print(f"\033[1mgarate\033[0m: {garate}")
print(f"\033[1mnegreturns\033[0m: {negreturns}, \033[1mSingleorMultiple\033[0m: {SingleorMultiple}")

if contains_positive == "yes" and contains_negative == "yes": 
    print(f"\nLetter Maker ran successfully. See files {ernum} Positive Investment Earnings VTD {vtd}.docx and {ernum} Negaitve Investment Earnings {vtd}.docx in current directory.")
else:
    print(f"\nLetter Maker ran successfully. See file {ernum} Summary Investment Earnings VTD {vtd}.docx in current directory.")
